# 27 Pandas Functions You Didn't Know Existed | P(Guarantee) = 0.9
## Making Pandas Even More Awesome
![](https://images.pexels.com/photos/5199661/pexels-photo-5199661.jpeg?auto=compress&cs=tinysrgb&dpr=2&h=650&w=940)
<figcaption style="text-align: center;">
    <strong>
        Photo by 
        <a href='https://www.pexels.com/@introspectivedsgn?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels'>Erik Mclean</a>
        on 
        <a href='https://www.pexels.com/photo/unrecognizable-man-in-panda-head-sitting-near-car-5199661/?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels'>Pexels.</a> All images are by the author unless specified otherwise.
    </strong>
</figcaption>

## Setup

In [1]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

warnings.filterwarnings("ignore")

## Introduction

## 1. `ExcelWriter`

`ExcelWriter` is a generic class for creating excel files (with sheets!) and writing DataFrames to them. Let's say we have these 2:

In [2]:
diamonds = sns.load_dataset("diamonds")
tips = sns.load_dataset("tips")

with pd.ExcelWriter("data/data.xlsx") as writer:
    diamonds.to_excel(writer, sheet_name="diamonds")
    tips.to_excel(writer, sheet_name="tips")

It has additional attributes to specify the datetime format to be used, whether you want to create a new excel file or modify an existing one, what happens when a sheet exists, etc. Check out the details from the [documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.ExcelWriter.html).

## 2. `factorize`

This function is a pandas alternative to Sklearn's `LabelEncoder`:

In [6]:
codes, unique = pd.factorize(diamonds["cut"], sort=True)

codes[:10]

array([0, 1, 3, 1, 3, 2, 2, 2, 4, 2], dtype=int64)

In [7]:
unique

CategoricalIndex(['Ideal', 'Premium', 'Very Good', 'Good', 'Fair'], categories=['Ideal', 'Premium', 'Very Good', 'Good', 'Fair'], ordered=False, dtype='category')

Pay attention to how the function returns the categories as well. 

In [10]:
diamonds["cut_enc"] = pd.factorize(diamonds["cut"])[0]
diamonds["cut_enc"].sample(5)

46228    0
10292    3
50925    1
38309    1
1775     0
Name: cut_enc, dtype: int64

## 3. `bdate_range`

A short-hand function to create TimeSeries indices with business-day frequency:

In [14]:
series = pd.bdate_range("2021-01-01", "2021-01-31")  # A period of one month
len(series)

21

Business-day frequencies are common in the financial world. So, this function may come in handy when reindexing existing time-series with `reindex` function.

## 4. `hasnans`

Pandas offers a quick method to check if a given series contains any nulls with `hasnans` attribute:

In [15]:
series = pd.Series([2, 4, 6, "sadf", np.nan])
series.hasnans

True

According to its [documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.hasnans.html), it enables various performance increases. Note that the attribute is pd.Series-only.

## 5. `convert_dtypes`

We all know that pandas has an annoying tendency to mark some columns as `object` data type. Instead of manually specifying their types, you can use `convert_dtypes` method which infers the best data type:

In [25]:
sample = pd.read_csv(
    "data/station_day.csv",
    usecols=["StationId", "CO", "O3", "AQI_Bucket"],
)
sample.dtypes

StationId      object
CO            float64
O3            float64
AQI_Bucket     object
dtype: object

In [28]:
sample.convert_dtypes().dtypes

StationId      string
CO            float64
O3            float64
AQI_Bucket     string
dtype: object

Unfortunately, it can't pares dates due to the caveats of different date time formats.

## 6. `at` and `iat`

These two accessors are much faster alternatives to `loc` and `iloc` with a disadvantage. They only allow selecting or replacing a single value at a time:

In [29]:
diamonds.at[234, "cut"]

'Ideal'

In [30]:
diamonds.iat[1564, 4]

61.2

In [31]:
# Replace 16541th row of the price column
diamonds.at[16541, "price"] = 10000

## 7. `min` and `max` along the columns axis

Even though `min` and `max` functions are well-known, they have another useful property for some edge-cases. Consider this dataset:

In [34]:
index = ["Diamonds", "Titanic", "Iris", "Heart Disease", "Loan Default"]
libraries = ["XGBoost", "CatBoost", "LightGBM", "Sklearn GB"]
df = pd.DataFrame(
    {lib: np.random.uniform(90, 100, 5) for lib in libraries}, index=index
)

df

,XGBoost,CatBoost,LightGBM,Sklearn GB
Diamonds,99.733903,96.540712,91.725201,93.640438
Titanic,93.130239,99.838026,93.586711,95.963391
Iris,97.153966,98.354851,90.585250,93.693119
Heart Disease,92.590182,94.402339,93.373349,93.399484
Loan Default,98.359192,91.073934,90.595855,92.900320


The above fake DataFrame is a point-performance of 4 different gradient boosting libraries on 5 datasets. We want to find the library that performed best at each dataset non-manually. Here is how you do it with `max`:

In [37]:
df.max(axis=1)

Diamonds         99.733903
Titanic          99.838026
Iris             98.354851
Heart Disease    94.402339
Loan Default     98.359192
dtype: float64

Just change the axis to 1 and you get a row-wise max/min. 

## 8. `pipe`

This is one of the best functions for doing data cleaning in a concise, compact manner. `pipe` function allows you to chain multiple custom functions into a single operation.

For example, let's say you have custom functions to `drop_duplicates`, `remove_outliers`, `encode_categoricals` that accept their own custom arguments. Here is how you apply all three in a single operation:

```python
df_preprocessed = (diamonds.pipe(drop_duplicates).
                            pipe(remove_outliers, ['price', 'carat', 'depth']).
                            pipe(encode_categoricals, ['cut', 'color', 'clarity'])
                  )
```

I like how this function resembles [Sklearn pipelines](https://towardsdatascience.com/how-to-use-sklearn-pipelines-for-ridiculously-neat-code-a61ab66ca90d). There is more you can do with it, so check out the [documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pipe.html) or this [helpful article](https://towardsdatascience.com/a-better-way-for-data-preprocessing-pandas-pipe-a08336a012bc).

## 9. `autocorr`

One of the critical components in time-series analysis is examining the autocorrelation of a variable. 

Autocorrelation is the plain-old correlation coefficient but it is calculated with the lagging version of a time series. 

In more detail, autocorrelation of a time series at `lag=k` is calculated as follows:

1. The time-series is shifted till `k` periods:

In [44]:
## HIDE
# Prep the data for an example
dt = pd.date_range("2021-01-01", periods=len(tips))
tips.index = dt

time_series = tips[["tip"]]

In [43]:
time_series["lag_1"] = time_series["tip"].shift(1)
time_series["lag_2"] = time_series["tip"].shift(2)
time_series["lag_3"] = time_series["tip"].shift(3)
time_series["lag_4"] = time_series["tip"].shift(4)
# time_series['lag_k'] = time_series['tip'].shift(k)

time_series.head()

,tip,lag_1,lag_2,lag_3,lag_4
2021-01-01,1.01,NaN,NaN,NaN,NaN
2021-01-02,1.66,1.01,NaN,NaN,NaN
2021-01-03,3.50,1.66,1.01,NaN,NaN
2021-01-04,3.31,3.50,1.66,1.01,NaN
2021-01-05,3.61,3.31,3.50,1.66,1.01


2. Correlation is calculated between the original `tip` and each `lag_*`. 

Instead of doing all this manually, you can use the `autocorr` function of Pandas:

In [47]:
# Autocorrelation of tip at lag_10
time_series["tip"].autocorr(lag=8)

0.07475238789967077

You can read more about the importance of autocorrelation in time-series analysis from this [post](https://towardsdatascience.com/advanced-time-series-analysis-in-python-decomposition-autocorrelation-115aa64f475e).

## 10. between

A rather nifty function for boolean indexing numeric features within a range:

In [53]:
# Get diamonds that are priced between 3500 and 3700 dollars
diamonds[diamonds["price"].between(3500, 3700, inclusive="neither")].sample(5)

,carat,cut,color,clarity,depth,table,price,x,y,z,cut_enc
4412,1.01,Very Good,I,SI1,58.8,58.0,3610,6.52,6.57,3.85,3
4806,0.90,Good,F,SI1,58.0,58.0,3699,6.24,6.28,3.63,2
4587,0.80,Ideal,F,VS2,62.0,55.0,3653,5.93,6.00,3.70,0
3972,0.73,Ideal,D,VS1,61.1,57.0,3509,5.85,5.81,3.56,0
4379,0.90,Premium,E,SI1,62.4,57.0,3609,6.19,6.14,3.85,1


## 11. `clip`

Outlier detection and removal is common in data analysis. 

`clip` function makes it really easy to find outliers outside a range and replacing them with the hard limits. For example, let's say we have a survey data collected from people aged 50-60. We want to check the age column and replace any values that are outside this range treating them as data entry mistakes:

In [55]:
ages = pd.Series([55, 52, 50, 66, 57, 59, 49, 60]).to_frame("ages")
ages

,ages
0,55
1,52
2,50
3,66
4,57
5,59
6,49
7,60


Two values are outside this range (49, 66). Let's fix it with `clip`:

In [57]:
ages.clip(50, 60)

,ages
0,55
1,52
2,50
3,60
4,57
5,59
6,50
7,60


Fast and efficient!

## 12. `nlargest` and `nsmallest`

Sometimes you don't just want the min/max of a column. You want to see the top N or ~(top N) values of a variable. This is where `nlargest` and `nsmallest` comes in handy.

Let's see top 5 most expensive and cheapest diamonds:

In [59]:
diamonds.nlargest(5, "price")

,carat,cut,color,clarity,depth,table,price,x,y,z,cut_enc
27749,2.29,Premium,I,VS2,60.8,60.0,18823,8.50,8.47,5.16,1
27748,2.00,Very Good,G,SI1,63.5,56.0,18818,7.90,7.97,5.04,3
27747,1.51,Ideal,G,IF,61.7,55.0,18806,7.37,7.41,4.56,0
27746,2.07,Ideal,G,SI2,62.5,55.0,18804,8.20,8.13,5.11,0
27745,2.00,Very Good,H,SI1,62.8,57.0,18803,7.95,8.00,5.01,3


In [60]:
diamonds.nsmallest(5, "price")

,carat,cut,color,clarity,depth,table,price,x,y,z,cut_enc
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43,0
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31,1
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31,2
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63,1
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75,2
